# Pound Drever Hall

## 0.0 Introduction

Pound–Drever–Hall (PDH) locking is a technique for extracting frequency offset information from a laser relative to a stable reference (e.g., a cavity). The method works by phase-modulating the laser beam to generate sidebands, which can be described using the Bessel expansion of a modulated sinusoid. The cavity reflects the carrier and sidebands differently, encoding frequency detuning into a phase shift of the reflected light. By demodulating this signal at the modulation frequency, we recover an error signal that is directly sensitive to frequency offset.

This notebook provides a visual introduction to these ideas before diving into the underlying math.

## 1.1 Sinusoid Modulation

The first thing we will do is create a time-domain sinusoid, this will represent our laser beam. We then modulate this sinusoid via an electro-optic modulator (EOM). The EOM will periodically phase-shift the signal at some given modulation frequency. Both the modulated and unmodulated carrier waves are visible below. 

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Generate time axis vectors
t1 = np.linspace(0, 1, 1000)
t2 = np.linspace(0, 10, 10000)

C = 3e8

freq = FloatSlider(value=4.4, min=0.1, max=10.0, step=0.1)
mod_freq = FloatSlider(value=0.9, min=0.1, max=10.0, step=0.1)
beta = FloatSlider(value=0.66, min=0.01, max=5.0, step=0.01)
phase_shift = FloatSlider(value=180, min=0, max=360, step=1)

refractive_index = FloatSlider(value=1.0, min=0.95, max=1.05, step=0.01)  # basically air
cav_freq = FloatSlider(value=15e9, min=13e9, max=17e9, step=10e6)  # around one FSR
amplitude_reflectivity = FloatSlider(value=0.95, min=0.8, max=1.0, step=0.01)


def sinusoid(freq, t, phase_shift_degrees=0):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + np.pi * phase_shift_degrees / 180) 
    return y

def modulated_sinusoid(freq, mod_freq, beta, t):
    omega = 2 * np.pi * freq
    y = np.cos(omega * t + beta * np.cos(2 * np.pi * mod_freq * t))
    return y

def get_sinusoid_spectrum(signal, t):
    N = len(t)
    dt = t[1] - t[0]
    Y = np.fft.fftshift(np.fft.fft(signal))
    f = np.fft.fftshift(np.fft.fftfreq(N, d=dt))
    Y_mag = np.abs(Y) / np.max(np.abs(Y))
    return f, Y_mag

def reflected_sinusoid(freq, mod_freq, beta, phase_shift, t):
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=t) + sinusoid(freq=freq, phase_shift_degrees=phase_shift, t=t)
    return y

def beat_signal(freq, mod_freq, beta, phase_shift, t):
    y1 = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t)
    y2 = sinusoid(freq=mod_freq, t=t)
    return y1 * y2

def beat_signal_vs_phase(freq, mod_freq, beta, t, n_points=361):
    phases = np.linspace(0, 360, n_points)
    means = []
    for phi in phases:
        y = beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phi, t=t)
        means.append(np.mean(y))
    return phases, np.array(means)

def beat_signal_vs_phase_normalized(freq, mod_freq, beta, t, n_points=361):
    phases = np.linspace(0, 360, n_points)
    means = []
    y_norm = np.mean(beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=180, t=t))
    for phi in phases:
        y = beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phi, t=t) - y_norm
        means.append(np.mean(y))
    return phases, np.array(means)


def ref_coefficient(frequency, cav_len, r, n):
    omega = frequency * 2 * np.pi
    fsr = C / (2*n*cav_len)
    z = np.exp(1j * omega / fsr)
    return r * (z - 1)/(1 - r**2 * z)


In [42]:
def plot_sinusoid(freq):
    T = t1
    y = sinusoid(freq=freq, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y, label=fr'cos(2pi * {freq}t)')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Unmodulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()

def plot_modulated_sinusoid(freq, mod_freq, beta):
    T = t1
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=T)
    plt.figure(figsize=(8,4))
    plt.plot(T, y, label=fr'cos(2pi * {freq} * t + {beta:.2e}*cos(2pi*{mod_freq:.2e}t))')
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Modulated Carrier")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_sinusoid, freq=freq)
interact(plot_modulated_sinusoid, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), Output()), _dom_classes=(…

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_modulated_sinusoid(freq, mod_freq, beta)>

If we look at the modulated signal we see something that seems similar to multiple sinusoidal signals with different frequencies superimposing themselves on each other. This can be proven with the Jacobi-Anger identity but for now we'll just grab the frequency spectrum and see what happens. 

In [43]:

def plot_spectrum_modulated_sinusoid(freq, mod_freq, beta):
    T = t2
    y = modulated_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, t=T)
    f, Y_mag = get_sinusoid_spectrum(signal=y, t=T)
    plt.figure(figsize=(10,5))
    plt.plot(f, Y_mag)
    plt.title("Frequency Spectrum of Phase-Modulated Signal")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized Magnitude")
    plt.xlim(freq-5*mod_freq, freq+5*mod_freq)
    plt.grid(True)
    plt.show()

interact(plot_spectrum_modulated_sinusoid, 
         freq=freq, 
         mod_freq=mod_freq, 
         beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_spectrum_modulated_sinusoid(freq, mod_freq, beta)>

We can see that for modulation frequencies less than the carrier frequency, and for comparatively small values of beta, we generally observe two prominent 'sidebands' equidistant on each side of the dominant band (which as you'd expect is the carrier frequency). For now we can assume that a properly designed PDH system will ensure that the modulation frequency is much lower than the carrier frequency and that our beta is fairly small.

## 1.2 Cavity interference
We can make the assumption that our modulating signal is very resistant to frequency drift. If we could find a way to translate frequency offset into a phase offset, we could then get a beat signal by superimposing the two fields and demodulating them with respect to our modulating tone. Taking the DC term of this signal would get us a value that is bijective (within our operating range) with the phase offset and thus the frequency offset. In other words, if the beat signal is odd-symmetric around our modulating tone, demodulating it with respect to that tone will allow us to obtain the magnitude and sign of our frequency offset. In other words, we should:

1. Impart a phase-shift on a signal with respect to frequency drift that is odd-symmetric
2. Make the phase shift as sharp as possible so we can detect it easily
3. Center it on our target carrier frequency that we are trying to correct to

We accomplish the three objectives above through either a Fabry-Perot cavity or a ring resonator. For what we care about they are functionally the same. We rotate the carrier with respect to frequency offset and demodulate the reflected field with respect to the modulation frequency.

For the code below, we assume a perfect cavity respose i.e. only the carrier component of our incident field is reflected. This is done to simplify the code. 

In [44]:
def plot_reflected_sinusoid(freq, mod_freq, beta, phase_shift):
    y = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t1)
    yf = reflected_sinusoid(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t2)
    f, Y_mag = get_sinusoid_spectrum(signal=yf, t=t2)
    
    plt.figure(figsize=(8,4))
    plt.plot(t1, y)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Reflected Field")
    plt.grid(True)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10,5))
    plt.plot(f, Y_mag)
    plt.title("Frequency Spectrum of Reflected Field")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Normalized Magnitude")
    plt.xlim(freq-5*mod_freq, freq+5*mod_freq)
    plt.grid(True)
    plt.show()

interact(plot_reflected_sinusoid, freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_reflected_sinusoid(freq, mod_freq, beta, phase_shift)>

## 1.3 Beat Signal

We can see that the time-domain representation of our reflected field takes on an interesting shape, and looking at the frequency-domain representation of that field we can see that for small phase offsets around a 180-degree baseline that the carrier tone is mostly eliminated (there are a couple reasons it doesn't disappear completely but that's not relevant for now). Now we can demodulate this reflection to obtain our beat signal. 

In [45]:
def plot_beat_signal(freq, mod_freq, beta, phase_shift):
    y = beat_signal(freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift, t=t1)

    dc_term = np.mean(y)
    print(f"Mean (DC component) of beat signal: {dc_term:.6f}")
    
    plt.figure(figsize=(8,4))
    plt.plot(t1, y, label="Beat signal")
    plt.axhline(dc_term, color="red", linestyle="--", label=f"Mean = {dc_term:.3e}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Beat Signal")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_beat_signal, freq=freq, mod_freq=mod_freq, beta=beta, phase_shift=phase_shift)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_beat_signal(freq, mod_freq, beta, phase_shift)>

We can see from the above that if our modulated signal is properly formed (two prominent sidebands), then small phase adjustments around the baseline rotation of 180 degrees results in a mean value that appears to be odd-symmetric and increasing/decreasing monotonically with respect to phase shift from baseline. The graph below plots this relationship out explicitly.  

In [46]:
def plot_beat_signal_vs_phase(freq, mod_freq, beta):
    phis, ys = beat_signal_vs_phase(freq=freq, mod_freq=mod_freq, beta=beta, t=t1)

    idx = np.argmin(np.abs(phis - 180))
    intercept_phase = phis[idx]
    intercept_value = ys[idx]

    plt.figure(figsize=(8,4))
    plt.plot(phis, ys, label="Mean beat signal")
    plt.axhline(0, color="black", linestyle="--", linewidth=0.8)

    plt.plot(intercept_phase, intercept_value, 'ro', label=f"180° intercept ({intercept_value:.3e})")
    plt.axvline(intercept_phase, color="red", linestyle="--", linewidth=0.8)

    plt.xlabel("Phase (Degrees)")
    plt.ylabel("DC Value")
    plt.title("Cavity Carrier Rotation vs Beat Signal DC Term")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_beat_signal_vs_phase, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_beat_signal_vs_phase(freq, mod_freq, beta)>

What we are looking at above can be thought of as a sort of toy representation of a PDH error signal. The basic shape is similar but a lot of factors present in a real-world implementation were not accounted for. We also haven't bothered to compute phase offset from frequency offset and display the error signal as a result of a frequency sweep, which would probably be more useful. Rather, we just assume that the cavity does its job properly and rotates our carrier term by some given amount depending on its frequency offset.

However, we can still see that this 'Toy Error Signal' is odd-symmetric around our 180-degree offset and is also monotonic in either direction up to a given point. Past this point the derivative changes signs and our error signal is no longer reliable (the bijection from frequency offset to beat signal DC term alluded to earlier breaks down). It's natural to ask if one should make sure to avoid excessive phase offsets (and by extension excessive frequency offsets). This is why servo controllers are combined with a separate acquisition strategy (e.g. sweep-and-hold, or an auxiliary wide-range actuator) to get the laser close enough that the PDH slope is reliable. 

We also see that there is generally some small offset at 180 degrees (cavity resonance). If we know what this offset, which can be obtained from the cavity response transfer function, we can normalize accordingly. 

In [47]:
def plot_beat_signal_vs_phase_normalized(freq, mod_freq, beta):
    phis, ys = beat_signal_vs_phase_normalized(freq=freq, mod_freq=mod_freq, beta=beta, t=t1)
    idx = np.argmin(np.abs(phis - 180))
    intercept_phase = phis[idx]
    intercept_value = ys[idx]
    plt.figure(figsize=(8,4))
    plt.plot(phis, ys, label="Mean beat signal")
    plt.axhline(0, color="black", linestyle="--", linewidth=0.8)
    plt.plot(intercept_phase, intercept_value, 'ro', label=f"180° intercept ({intercept_value:.3e})")
    plt.axvline(intercept_phase, color="red", linestyle="--", linewidth=0.8)
    plt.xlabel("Phase (Degrees)")
    plt.ylabel("DC Value")
    plt.title("Cavity Carrier Rotation vs Normalized Beat Signal DC Term")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_beat_signal_vs_phase_normalized, freq=freq, mod_freq=mod_freq, beta=beta)

interactive(children=(FloatSlider(value=4.4, description='freq', max=10.0, min=0.1), FloatSlider(value=0.9, de…

<function __main__.plot_beat_signal_vs_phase_normalized(freq, mod_freq, beta)>

## 2.1 Bessel Expansion

As mentioned earlier, PDH exploits the frequency domain characteristics of a modulated sinusoid, these characteristics are visible in the Bessel function expansion of the signal via the Jacobi-Anger identity. 


$$
e^{i\beta \cos(\omega_mt)} = \sum_{n=-\infty}^\infty i^nJ_n(\beta)e^{in\omega_mt}
$$

Given our modulated sinusoid signal, as visible from the code:

1.

$$
cos(\omega t + \beta \cos(\omega_mt)) = \mathbb{R}\{e^{i\omega t}e^{i\beta \cos(\omega_m t)}\} = \mathbb{R}\{e^{i\omega t}\sum_{n=-\infty}^\infty i^nJ_n(\beta)e^{in\omega_m t}\}
$$

Where

$$
J_n(\beta) = \frac{1}{2\pi}\int_0^{2\pi}\cos(n\theta - \beta \sin(\theta))d\theta
$$

It's relatively easy to see from the integral above that for small values of beta and large values of n, the overall magnitude will be quite low. Integrating a high-frequency sinusoid across a large number of cycles will tend to yield a very small average value. The beta scaled sinusoidal offset can counter this by introducing a DC term into the resultant signal's frequency spectrum but for small values of beta this DC term is negligible. In other words, we can say that for small values of beta we only really care about the first two terms $J_0$ and $J_1$

Evaluating 1. we get:

$$
cos(\omega t + \beta \cos(\omega_m t)) \approx J_0(\beta)\cos(\omega t) - J_1(\beta)[\sin((\omega + \omega_m)t) - \sin((\omega - \omega_m)t)]
$$

Where:
$$
J_0(\beta) \approx 1 - \frac{\beta^2}{4} \newline J_1(\beta) \approx \frac{\beta}{2}
$$


What all this means is that if our beta is small, when we take the frequency spectrum of our signal we should see three main contributions present. One large one at the carrier frequency and two smaller ones at the carrier frequency plus or minus the modulation frequency. These are the sidebands. This is easily verifiable by playing around with beta in the frequency spectrum plots above. When beta is made larger the other Bessel functions are able to contribute more and the frequency spectrum acquires more spikes. We don't want this so we make sure beta is small.


## 2.2 Real PDH signal
We have our current toy PDH error signal but it is not really reflective of what a real PDH error signal looks like beyond the general idea of being odd-symmetric around a central point. The next step is to get a realistic signal.

We define our fields as follows:

$$
E_{inc} = E_0e^{i\omega t}\newline
E_{ref} = E_1e^{i\omega t}
$$
Where $E_0$ and $E_1$ are complex constants in order to account for phase differences between the two signals.


Assuming a symmetric Fabry-Perot cavity with no losses, the reflection coefficient can be obtained as follows:
$$
F(\omega) = \frac{E_{ref}}{E_{inc}} = \frac{r(e^{i\frac{\omega}{\Delta v_{fsr}}}-1)}{1-r^2e^{i\frac{\omega}{\Delta v_{fsr}}}}
$$

Where $r$ is the amplitude reflection coefficient and $\Delta v_{fsr} = \frac{c}{2L}$ is the free spectral range (distance between modes) for a cavity with length $L$.


We can think of the reflection coefficient as a complex-valued vector with a phase angle dependent on the carrier frequency. The reflected field is then simply the incident field stretched and rotated in the complex plane by this reflection coefficient vector. Most of the time the reflection coefficient vector is relatively real-valued and constant magnitude. However, when approaching resonance it becomes almost entirely imaginary and flips over the real axis from +-i to -+i during resonance. At resonance the magnitude is zero. This sign flip also means that the cavity response is odd-symmetric about resonance. Below is a visualization:

In [48]:
SPAN = 1.2
N = 4001

def plot_ref_coefficient(frequency, r, n):
    cav_len = 0.01
    F = ref_coefficient(frequency=frequency, r=r, n=n, cav_len=cav_len)

    plt.figure(figsize=(6,6))
    plt.quiver(0, 0, F.real, F.imag,
               angles='xy', scale_units='xy', scale=1,
               color='b', label=f'F(f), |F|={abs(F):.2f}')

    plt.axhline(0, color='k', lw=0.5)
    plt.axvline(0, color='k', lw=0.5)
    plt.gca().set_aspect('equal', 'box')

    plt.xlim(-1.2, 1.2)
    plt.ylim(-1.2, 1.2)

    plt.title('Fabry-Pérot Reflection Coefficient F(f)')
    plt.xlabel('Re{F}')
    plt.ylabel('Im{F}')
    plt.legend()
    plt.grid(True)

    fsr = C / (2*n*cav_len)
    nu = np.linspace(-SPAN*fsr/2, SPAN*fsr/2, N)
    x = nu/fsr
    phi = (2 * np.pi * x)
    z = np.exp(1j * phi)
    K = (r * (z - 1)) / (1 - (r**2) * z)
    mag = np.abs(K)
    phase = np.unwrap(np.angle(K))  # radians

    plt.figure(figsize=(10,4))
    plt.plot(x, mag, lw=2)
    plt.xlabel('Detuning f / FSR')
    plt.ylabel('|F(f)|')
    plt.title('Reflected Amplitude |F(f)| vs Detuning')
    plt.grid(True)

    plt.figure(figsize=(10,4))
    plt.plot(x, phase, lw=2)
    plt.xlabel('Detuning f / FSR')
    plt.ylabel('arg F(f)')
    plt.title('Reflected Phase arg F(f) vs Detuning')
    plt.grid(True)

    plt.show()


interact(plot_ref_coefficient, frequency=cav_freq, r=amplitude_reflectivity, n=refractive_index)

interactive(children=(FloatSlider(value=15000000000.0, description='frequency', max=17000000000.0, min=1300000…

<function __main__.plot_ref_coefficient(frequency, r, n)>

As we detune the frequency, the reflection coefficient $F(\omega)$ traces a circle on the left side of the complex plane, consistent with the literature. If we set $r=1$, this behavior disappears. This is because the input mirror is a perfect reflector, no leakage field exists, and the total reflection is a constant $\pi$-phase flip $F(\omega) = -1$ (purely real, no frequency dependence).  

For $r < 1$ (symmetric, lossless cavity), the interference between the prompt and leakage fields drives $|F|$ to zero at resonance, and $\arg F$ flips sign across resonance. This is exactly the odd-symmetric response PDH exploits.